In [1]:
import random
import copy

In [2]:
def initialize(number_of_variables): # inicijalizacija pocetne valuacije
    u = []
    for i in range(number_of_variables):
        if random.random() < 0.5:
            u.append(True)
        else:
            u.append(False)
    return u

In [3]:
def evaluate_truth_assignment(u, c): # evaluacija trenutne valuacije, broji se koliko klauza je zadovoljeno tom valuacijom, ovo je funkcija cilja koja se maksimizuje
    num_satisfied = 0

    for clause in c:
        satisfied = False
        
        for literal in clause:
            if literal > 0:
                if(u[literal - 1]):
                    satisfied = True
            else:
                literal = -literal
                if(not (u[literal - 1])):
                    satisfied = True

        if(satisfied):
            num_satisfied = num_satisfied + 1

    return num_satisfied

In [4]:
def generate_new_assignment(u): # u svakoj iteraciji se pravi nova valuacija od prethodne, tako sto se jedna promenljiva promeni iz True u False ili obrnuto
    index = random.randrange(len(u))
    u[index] = not (u[index])
    return index

In [5]:
def bf_generate_assignment(number_of_variables, bf_iteration): # generisanje valuacija u brute force algoritmu
    u = []
    a = bf_iteration
    for i in range(number_of_variables):
        u.append(a % 2 == 1)
        a = a // 2
    return u

In [6]:
def brute_force(number_of_variables, c): # brute force algoritam
    total_attempts = 2 ** number_of_variables
    best = 0
    best_assignment = bf_generate_assignment(number_of_variables, 0)

    for i in range(total_attempts):
        u = bf_generate_assignment(number_of_variables, i)
        eval = evaluate_truth_assignment(u, c)
        if eval > best:
            best = eval
            best_assignment = copy.deepcopy(u)

    return best_assignment, best

In [7]:
def simulated_annealing(number_of_variables, c, iterations): # simulirano kaljenje za aproksimaciju maksimalne zadovoljivosti
    u = initialize(number_of_variables) # inicijalizacija pocetne valuacije

    current_optimal_evaluation = evaluate_truth_assignment(u, c)
    global_optimal_evaluation = current_optimal_evaluation
    current_best_assignment = copy.deepcopy(u)
    global_best_assignment = copy.deepcopy(u) # pocetna valuacija je jedina koju smo razmatrali, tako da je za sada najbolja

    for i in range(1, iterations+1): # posto u racunanju p delimo sa korenom iz i, i mora da krene od 1
        changed_index = generate_new_assignment(u) # kreira se nova valuacija, pamti se indeks gde je bila izmena u slucaju da je resenje losije
        new_evaluation = evaluate_truth_assignment(u, c) # izracunavanje funkcije cilja

        if new_evaluation > current_optimal_evaluation: # ako je za trenutno resenje funkcija cilja u lokalnom maksimumu, cuva se kao lokalno najbolje
            current_optimal_evaluation = new_evaluation
            current_best_assignment = copy.deepcopy(u)
            if new_evaluation > global_optimal_evaluation: # ako je za trenutno resenje funkcija cilja u globalnom maksimumu, cuva se kao globalno najbolje
                global_optimal_evaluation = new_evaluation
                global_best_assignment = copy.deepcopy(u)
        else: # racunanje p i q direktno preuzeto iz koda sa vezbi, koristi se takodje algoritam simuliranog kaljenja, ali za drugi problem
            # (https://github.com/MATF-RI/Materijali-sa-vezbi/blob/master/2021_2022/live/03_s_metaheuristics.ipynb)
            # ponekad se nastavlja iz resenja koje nije najbolje da bi se izaslo iz lokalnog maksimuma, ali je verovatnoca za to sve manja svakom sledecom iteracijom
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q: # prelazak u novo resenje iako nije bolje
                current_optimal_evaluation = new_evaluation
                current_best_assignment = copy.deepcopy(u)
            else: # povratak u prethodno
                u[changed_index] = not u[changed_index]

    return global_best_assignment, global_optimal_evaluation # algoritam vraca najbolju valuaciju i broj klauza zadovoljen najboljom pronadjenom valuacijom

In [ ]:
dimacsfile = open("random_dimacs5.txt", "r") # ulaz programa je dimacs fajl, pretpostavlja se da je sadrzaj fajla ispravan i da nema komentare

startline = dimacsfile.readline()
args = startline.split()
number_of_variables = int(args[2]) # izdvajaju se broj iskaznih promenljivih i broj klauza
number_of_clauses = int(args[3])

c = [] # lista klauza

print("Number of variables: " + str(number_of_variables))
print("Number of clauses: " + str(number_of_clauses))

for i in range(number_of_clauses): # ucitavanje klauza iz dimacs fajla
    current_clause = []
    current_clause_string = dimacsfile.readline()
    list_of_literals = current_clause_string.split()
    for literal in list_of_literals:
        if(not (literal == '0')):
            current_clause.append(int(literal))
        else:
            break
    c.append(current_clause)

print("Clauses: " + str(c))

dimacsfile.close()

In [403]:
u_best, max_satisfied_clauses = simulated_annealing(number_of_variables, c, 8000) # pokretanje algoritma simuliranog kaljenja i ispis rezultata

print("Best truth assignment: " + str(u_best))
print("Maximum number of satisfied clauses: " + str(max_satisfied_clauses))

In [ ]:
# kod ispod se pokrece samo ako se brute force algorithm moze izvrsiti u razumnom vremenu

u_best, max_satisfied_clauses = brute_force(number_of_variables, c)

print("Best truth assignment according to brute force: " + str(u_best))
print("Maximum number of satisfied clauses according to brute force: " + str(max_satisfied_clauses))